In [1]:
import numpy as np
import math
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit

np.seterr(all='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
alterar='20200908'

In [3]:
bd = pd.read_csv('data/dados_gerais_'+alterar+'.csv',sep=",") #A variável de interesse tem que estar nomeada como "Confirmed"
bd1 = pd.read_csv('data/dados_nacionais_'+alterar+'.csv',sep=",") #A variável de interesse tem que estar nomeada como "Confirmed"
bd2 = pd.read_csv('data/dados_mundiais_'+alterar+'.csv',sep=",") #A variável de interesse tem que estar nomeada como "Confirmed"

bd["preinflex"]=0
bd["peak"]=0
bd["postinflex"]=0

## Aplicando modelo pontencial exponencial para prever pontos de inflexão, pico e dia de término da pandemia

In [4]:
country=bd["Country/Region"].unique()
for s in country: 
    _f1 = bd["Country/Region"] == s
    data=bd[_f1]
    data=data.loc[:,"Confirmed"].reset_index(drop= True)
    data=data.values 

    n = len(data)
    g = data.cumsum()
    M = data.max()
    t = np.where(data == data.max())[0]#[0]+1
    tM=np.where(data == data.max())[0][len(t)-1]+1
    tM1 = round(tM-0.15*n)
    tM2 = round(tM+0.15*n)
    _sum = data.sum()

    f = np.empty(n)
    mloss = 1e90
    itr = 0

    for c in np.arange(-0.05, -1.51, -0.05):
        start, end = -c*tM1, (-c*tM2)+0.1
        for b in np.arange(start, end, 0.1):
            for i in range(n):
                f[i] = np.power(i+1, b) * np.exp(c*(i+1))
            a = _sum / f.sum()
            F = a * f
            loss = np.linalg.norm(data-F)
            if loss < mloss:
                besta = a
                bestb = b
                bestc = c
                bestarea = np.around(F.sum(), 2)
                mloss = loss;
                itr += 1
                F2plt = F

    bd.loc[_f1,"preinflex"]+=round(-(bestb-math.sqrt(math.fabs(bestb)))/bestc ) 
    bd.loc[_f1,"peak"] +=round(-bestb/bestc)
#     bd.loc[_f1,"cases_day"] += round(besta*(np.power((-bestb/bestc), bestb))*np.exp(bestc*(-bestb/bestc)))
    bd.loc[_f1,"postinflex"] += round(-(bestb+math.sqrt(math.fabs(bestb)))/bestc)

## Definindo estágio atual da pandemia por país

In [5]:
df=bd
country=df["Country/Region"].unique()
for s in country: 
    _f1 = df["Country/Region"] == s
    data=df[_f1]
    data=len(data.loc[next(x for x in data[data["Confirmed"]>0].index):,"Confirmed"].reset_index(drop= True))
    if data <= df.loc[_f1,"preinflex"].values[0]:
        df.loc[_f1,"estagio"]=1
    elif data<df.loc[_f1,"peak"].values[0]:
        df.loc[_f1,"estagio"]=2
    elif data==df.loc[_f1,"postinflex"].values[0]:
        df.loc[_f1,"estagio"]=3
    elif data<=df.loc[_f1,"postinflex"].values[0]:
        df.loc[_f1,"estagio"]=4
    else: df.loc[_f1,"estagio"]=5

## Dados para modelagem de notas

In [6]:
df_notas=df.dropna(subset=["populacao","day_of_week"]).reset_index(drop=True)
df_notas=df_notas.drop(columns=["estagio","preinflex","postinflex","peak"])
df_notas.to_csv('./datasets/dados_notas_'+alterar+'.csv', index=False)

In [7]:
df_casos=df.drop(columns=["day_of_week", "Qtde Autorizados", "Valor Autorizados", "populacao","Qtde Autorizados_2019_mean","Valor Autorizados_2019_mean","preinflex","postinflex","peak"])
df_casos1=df_casos.drop(columns=["estagio"])
df_casos1=df_casos1.dropna()
df_casos1.to_csv('./datasets/complete_mundial_estadual_model_'+alterar+'.csv',index=False)

In [8]:
df_all=df.loc[:,["Country/Region","estagio"]].drop_duplicates()

estagio2= df_all["Country/Region"][df_all["estagio"]==2]
estagio1= df_all["Country/Region"][df_all["estagio"]==1]
estagio4= df_all["Country/Region"][df_all["estagio"]==4]
estagio5= df_all["Country/Region"][df_all["estagio"]==5]
estagio3= df_all["Country/Region"][df_all["estagio"]==3]

n=len(df_all)
n2=len(estagio2)
n1=len(estagio1)
n4=len(estagio4)
n5=len(estagio5)
n3=len(estagio3)
p2=round(n2*100/n)
p3=round(n3*100/n)
p1=round(n1*100/n)
p4=round(n4*100/n)
p5=round(n5*100/n)

print(f'Do total de {n} países:')
print(f'{n1} ({p1}%) estão no estágio 1 da pandemia')
print(f'{n2} ({p2}%) estão no estágio 2 da pandemia')
print(f'{n3} ({p3}%) estão no estágio 3 da pandemia')
print(f'{n4} ({p4}%) estão no estágio 4 da pandemia')
print(f'{n5} ({p5}%) estão no estágio 5 da pandemia')


Do total de 200 países:
1 (0%) estão no estágio 1 da pandemia
21 (10%) estão no estágio 2 da pandemia
2 (1%) estão no estágio 3 da pandemia
40 (20%) estão no estágio 4 da pandemia
136 (68%) estão no estágio 5 da pandemia


## Dados para modelagem de casos

In [9]:
if n1<2:
    df_casos=df_casos[df_casos["estagio"]!=1]
if n2<2:
    df_casos=df_casos[df_casos["estagio"]!=2]
if n3<2:
    df_casos=df_casos[df_casos["estagio"]!=3]
if n4<2:
    df_casos=df_casos[df_casos["estagio"]!=4]
if n5<2:
    df_casos=df_casos[df_casos["estagio"]!=5]

dados_sinteticos_filtrado=df_casos.copy()
# dados_sinteticos_filtrado=df_casos[df_casos["estagio"]!=3]

intervalo = (dados_sinteticos_filtrado['Country/Region'] == 'Spain') | (dados_sinteticos_filtrado['Country/Region'] == 'Brazil')|(dados_sinteticos_filtrado['Country/Region'] == 'United States')| (dados_sinteticos_filtrado['Country/Region'] == 'China')
intervalo1= (dados_sinteticos_filtrado['Country/Region'] == 'BA') | (dados_sinteticos_filtrado['Country/Region'] == 'RJ')
intervalo2=(dados_sinteticos_filtrado['Country/Region'] == 'ES')| (dados_sinteticos_filtrado['Country/Region'] == 'AL')
intervalo3=(dados_sinteticos_filtrado['Country/Region'] == 'AC')| (dados_sinteticos_filtrado['Country/Region'] == 'PB')| (dados_sinteticos_filtrado['Country/Region'] == 'PI')| (dados_sinteticos_filtrado['Country/Region'] == 'RN')|(dados_sinteticos_filtrado['Country/Region'] == 'RO')| (dados_sinteticos_filtrado['Country/Region'] == 'RR')| (dados_sinteticos_filtrado['Country/Region'] == 'RS')| (dados_sinteticos_filtrado['Country/Region'] == 'SE')

dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo].index, inplace=True)
dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo1].index, inplace=True)
dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo2].index, inplace=True)
dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo3].index, inplace=True)
dados_sinteticos_filtrado = dados_sinteticos_filtrado.reset_index(drop=True)

/home/lucas/miniconda3/envs/covid-web/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/lucas/miniconda3/envs/covid-web/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/lucas/miniconda3/envs/covid-web/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [10]:
banco=dados_sinteticos_filtrado[["Country/Region","estagio"]].drop_duplicates().reset_index(drop=True)
regions_info = banco.groupby('Country/Region').max()

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=0)

regions = list(regions_info.index)
classes = list(regions_info['estagio'])

for train_index, test_index in sss.split(regions, classes):
#     print(len(test_index))
    regions_train, regions_test = regions_info.iloc[train_index,:], regions_info.iloc[test_index,:]


In [11]:
regions_test=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_test.index))]

regions_train=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_train.index))]

regions_test = pd.concat([regions_test,  df_casos[df_casos['Country/Region'] == 'Brazil']],axis=0)
regions_test = pd.concat([regions_test,  df_casos[df_casos['Country/Region'] == 'Spain']],axis=0)
regions_test = pd.concat([regions_test,  df_casos[df_casos['Country/Region'] == 'BA']],axis=0)
regions_test = pd.concat([regions_test,  df_casos[df_casos['Country/Region'] == 'RJ']],axis=0)
regions_test=regions_test.reset_index(drop=True)

In [12]:
df_all=regions_train.loc[:,["Country/Region","estagio"]].drop_duplicates()

estagio2= df_all["Country/Region"][df_all["estagio"]==2]
estagio1= df_all["Country/Region"][df_all["estagio"]==1]
estagio4= df_all["Country/Region"][df_all["estagio"]==4]
estagio5= df_all["Country/Region"][df_all["estagio"]==5]
estagio3= df_all["Country/Region"][df_all["estagio"]==3]

n=len(df_all)
n2=len(estagio2)
n1=len(estagio1)
n4=len(estagio4)
n5=len(estagio5)
n3=len(estagio3)
p2=round(n2*100/n)
p3=round(n3*100/n)
p1=round(n1*100/n)
p4=round(n4*100/n)
p5=round(n5*100/n)

print(f'Do total de {n} países:')
print(f'{n1} ({p1}%) estão no estágio 1 da pandemia')
print(f'{n2} ({p2}%) estão no estágio 2 da pandemia')
print(f'{n3} ({p3}%) estão no estágio 3 da pandemia')
print(f'{n4} ({p4}%) estão no estágio 4 da pandemia')
print(f'{n5} ({p5}%) estão no estágio 5 da pandemia')

Do total de 128 países:
0 (0%) estão no estágio 1 da pandemia
14 (11%) estão no estágio 2 da pandemia
1 (1%) estão no estágio 3 da pandemia
26 (20%) estão no estágio 4 da pandemia
87 (68%) estão no estágio 5 da pandemia


In [13]:
if n1<2:
    regions_train=regions_train[regions_train["estagio"]!=1]
if n2<2:
    regions_train=regions_train[regions_train["estagio"]!=2]
if n3<2:
    regions_train=regions_train[regions_train["estagio"]!=3]
if n4<2:
    regions_train=regions_train[regions_train["estagio"]!=4]
if n5<2:
    regions_train=regions_train[regions_train["estagio"]!=5]

In [14]:
banco=regions_train[["Country/Region","estagio"]].drop_duplicates().reset_index(drop=True)
regions_info = banco.groupby('Country/Region').max()

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.50, train_size=0.50, random_state=0)

regions = list(regions_info.index)
classes = list(regions_info['estagio'])

for train_index, val_index in sss.split(regions, classes):
#     print(len(test_index))
    regions_train, regions_val = regions_info.iloc[train_index,:], regions_info.iloc[val_index,:]

In [15]:
regions_train=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_train.index))]
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'China']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'United States']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'AC']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'PB']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'PI']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'RN']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'RO']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'RS']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'RR']],axis=0)
regions_train = pd.concat([regions_train,  df_casos[df_casos['Country/Region'] == 'PE']],axis=0)
regions_train=regions_train.reset_index(drop=True)

In [16]:
regions_val=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_val.index))]
regions_val = pd.concat([regions_val,  df_casos[df_casos['Country/Region'] == 'AL']],axis=0)
regions_val = pd.concat([regions_val,  df_casos[df_casos['Country/Region'] == 'ES']],axis=0)
regions_val=regions_val.reset_index(drop=True)

In [17]:
df_all=regions_val.loc[:,["Country/Region","estagio"]].drop_duplicates()

estagio2= df_all["Country/Region"][df_all["estagio"]==2]
estagio1= df_all["Country/Region"][df_all["estagio"]==1]
estagio4= df_all["Country/Region"][df_all["estagio"]==4]
estagio5= df_all["Country/Region"][df_all["estagio"]==5]
estagio3= df_all["Country/Region"][df_all["estagio"]==3]

n=len(df_all)
n2=len(estagio2)
n1=len(estagio1)
n4=len(estagio4)
n5=len(estagio5)
n3=len(estagio3)
p2=round(n2*100/n)
p3=round(n3*100/n)
p1=round(n1*100/n)
p4=round(n4*100/n)
p5=round(n5*100/n)

print(f'Do total de {n} países:')
print(f'{n1} ({p1}%) estão no estágio 1 da pandemia')
print(f'{n2} ({p2}%) estão no estágio 2 da pandemia')
print(f'{n3} ({p3}%) estão no estágio 3 da pandemia')
print(f'{n4} ({p4}%) estão no estágio 4 da pandemia')
print(f'{n5} ({p5}%) estão no estágio 5 da pandemia')

Do total de 66 países:
0 (0%) estão no estágio 1 da pandemia
7 (11%) estão no estágio 2 da pandemia
0 (0%) estão no estágio 3 da pandemia
13 (20%) estão no estágio 4 da pandemia
46 (70%) estão no estágio 5 da pandemia


## Salvando treinamento,validação e teste-casos

In [18]:
regions_val=regions_val.dropna()
regions_test=regions_test.dropna()
regions_train=regions_train.dropna()

regions_train.to_csv('./datasets/train_mundial_estadual_model_'+alterar+'.csv',index=False)
regions_test.to_csv('./datasets/test_mundial_estadual_model_'+alterar+'.csv',index=False)
regions_val.to_csv('./datasets/val_mundial_estadual_model_'+alterar+'.csv',index=False)

## Dados para modelagem pico

In [19]:
df_pico=df.drop(columns=["day_of_week", "Qtde Autorizados", "Valor Autorizados", "populacao","Qtde Autorizados_2019_mean","Valor Autorizados_2019_mean"])
df_pico1=df_pico.drop(columns=["estagio"])
df_pico1=df_pico1.dropna()
df_pico1.to_csv('./datasets/complete_mundial_estadual_model_peak_'+alterar+'.csv',index=False)

In [20]:
dados_sinteticos_filtrado=df_pico.copy()
intervalo = ((dados_sinteticos_filtrado['estagio'] == 1) | (dados_sinteticos_filtrado['estagio'] == 2) | (dados_sinteticos_filtrado['estagio'] == 3))
dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo].index, inplace=True)
dados_sinteticos_filtrado = dados_sinteticos_filtrado.reset_index(drop=True)

In [21]:
intervalo1 = ((dados_sinteticos_filtrado['Country/Region'] == "Spain")|(dados_sinteticos_filtrado['Country/Region'] == "Italy"))
dados_sinteticos_filtrado.drop(dados_sinteticos_filtrado[intervalo1].index, inplace=True)
dados_sinteticos_filtrado = dados_sinteticos_filtrado.reset_index(drop=True)

In [22]:
banco=dados_sinteticos_filtrado[["Country/Region","estagio"]].drop_duplicates().reset_index(drop=True)
regions_info = banco.groupby('Country/Region').max()

In [23]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, train_size=0.90, random_state=1)

regions = list(regions_info.index)
classes = list(regions_info['estagio'])

for train_index, test_index in sss.split(regions, classes):
#     print(len(test_index))
    regions_train, regions_test = regions_info.iloc[train_index,:], regions_info.iloc[test_index,:]

In [24]:
regions_test=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_test.index))]

regions_train=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_train.index))]

In [25]:
regions_test = pd.concat([regions_test,  df_pico[df_pico['Country/Region'] == 'Italy']],axis=0)
regions_test = pd.concat([regions_test,  df_pico[df_pico['Country/Region'] == 'Spain']],axis=0)
# regions_test=pd.concat([regions_test,dadosBR_test],axis=0, sort=True)
regions_test=regions_test.reset_index(drop=True)


In [26]:
banco=regions_train[["Country/Region","estagio"]].drop_duplicates().reset_index(drop=True)
regions_info = banco.groupby('Country/Region').max()


In [27]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.50, train_size=0.50, random_state=0)

regions = list(regions_info.index)
classes = list(regions_info['estagio'])

for train_index, val_index in sss.split(regions, classes):
#     print(len(test_index))
    regions_train, regions_val = regions_info.iloc[train_index,:], regions_info.iloc[val_index,:]


In [28]:
regions_val=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_val.index))]

regions_train=dados_sinteticos_filtrado[dados_sinteticos_filtrado["Country/Region"].isin(list(regions_train.index))]


In [29]:
regions_val=regions_val.dropna()
regions_test=regions_test.dropna()
regions_train=regions_train.dropna()

regions_train.to_csv("./datasets/train_mundial_estadual_model_peak_"+alterar+".csv",index=False)
regions_test.to_csv("./datasets/test_mundial_estadual_model_peak_"+alterar+".csv",index=False)
regions_val.to_csv("./datasets/val_mundial_estadual_model_peak_"+alterar+".csv",index=False)